In [1]:
import pickle
import pandas as pd
from sklearn import preprocessing
import re
import scipy
import numpy as np
import glob
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

2022-05-11 23:27:51.246067: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
filename = "22-04-02_23-10-18_doc2vec_Vect_WoRul_Max-Len-Seq-20_Freq-20_CNN"
path = "F:/RweetMiner/RweetMinerNew/request_identification_class_DLCorrected/doc2vec_clfModels/" + filename
load_model = pickle.load(open(path, 'rb'))

In [80]:

class UsePretrainedVectClf:
    def __init__(self, pathToTokenizer, pathToClfModel, path_to_PreprocessedDataset, path_to_nonPreprocessedDataset, include_rules = False):
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
        self.path_to_PreprocessedDataset = path_to_PreprocessedDataset
        self.include_rules = include_rules
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
    
    def traditional_ChangeVectPath(self, pathToTokenizer):
        self.pathToTokenizer = pathToTokenizer   
        
    def traditional_loadClf(self):  
        clf_model = pickle.load(open(self.pathToClfModel, 'rb'))
        return clf_model
    
    def traditional_ChangeClfModelPath(self, pathToClfModel):
        self.pathToClfModel = pathToClfModel     
        
    def traditional_loadVect(self):
        Vect = pickle.load(open(self.pathToTokenizer, 'rb'))
        return Vect
    
    def apply_gen_rules_features_ngrams(self, X, X_train_WoR_dtm, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_ngrams(X)
        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_features


    def gen_rules_features_ngrams(self, X_data_series):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
        it as features...
        I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
          for changing datatypes'''
        
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            columnName = "RegEx_" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_ngrams(text, regex))
        temp_sparse = scipy.sparse.csr_matrix(temp.values)
        return temp_sparse, features_arg
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def apply_regex_ngrams(self, text, regex):
        match_found = (re.search(regex, text) != None)
        match_found = int(match_found == True)
        return match_found
    
    def traditional_getFeatures(self, Vect):
        df = pd.read_csv(self.path_to_PreprocessedDataset, encoding = "ISO-8859-1")
        X_train_WoR_dtm = Vect.transform(df["tweet_text"])
        X_train_Features = list(Vect.get_feature_names_out())
        if self.include_rules:
            df_notPreprocessed = pd.read_csv(self.path_to_nonPreprocessedDataset,  encoding = "ISO-8859-1")
            X_train_dtm, X_train_Features = self.apply_gen_rules_features_ngrams(df_notPreprocessed["tweet_text"], \
                                                                      X_train_WoR_dtm, X_train_Features)
        else:
            X_train_dtm = X_train_WoR_dtm
        return X_train_dtm.toarray(), Vect


    
    def get_predictions(self, clf_model, X):
        return clf_model.predict(X)
        
    
    def evaluation(self):
        if "WRul" in self.pathToTokenizer:
            if not self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WoRul" in self.pathToTokenizer:
            if self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WRul" in self.pathToClfModel:
            if not self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WoRul" in self.pathToClfModel:
            if self.include_rules:
                print("Vectoirzer and appending does not match")
        if not self.pathToClfModel.split("/")[-1].split("_")[5:-1] == self.pathToTokenizer.split("/")[-1].split("_"):
            print("Vectorizer and classifer does not match.")
            print("Vectorizer:", self.pathToTokenizer)
            print("Classifier:", self.pathToClfModel)
            print("Classifier:", self.pathToTokenizer.split("/")[-1].split("_"), "Vectorizer:" , self.pathToClfModel.split("/")[-1].split("_")[5:-1])
def generateNameFromDataSetName(list1):
    str1 = ""
    for e in list1:
        if e == "":
            str1 += '.'.join(str(e))
        else:
            if e == "csv":
                e = "_withPredictions." + e
                str1 += ''.join(str(e))
            else:
                str1 += ''.join(str(e))
    return ".." + str1
  
   

In [81]:
# # vect_filename = "22-05-06_20-59-32_Unigrams_CountVect_WRul_Freq-3803"



# pathToTokenizer= "" 

# # clfModel_filename = "22-05-06_20-59-32_CountVect_WRul_Freq-3803_22-05-06_20-59-32_Unigrams_CountVect_WRul_Freq-3803_DT"
# # clfModel_filename = ""

pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../simple_vect/"
pathToClfFolder = "../simple_clfModels/"




obj = UsePretrainedVectClf(pathToVectFolder, pathToClfModel, path_to_PreprocessedDataset, path_to_nonPreprocessedDataset, include_rules = False)
NotProcessedDataWithPredictions = pd.read_csv(obj.path_to_nonPreprocessedDataset, encoding = "ISO-8859-1")
ProcessedDataWithPredictions = pd.read_csv(obj.path_to_PreprocessedDataset, encoding = "ISO-8859-1")
# print(NotProcessedDataWithPredictions.shape, ProcessedDataWithPredictions.shape)
for vect_filename in glob.glob(pathToVectFolder + "/*"):
    print(40*"+")
    obj.traditional_ChangeVectPath(vect_filename)
    Vect = obj.traditional_loadVect()
    for clfModel_filename in glob.glob(pathToClfFolder + "*"):
        if clfModel_filename.split("/")[-1].split("_")[5:-1] == vect_filename.split("/")[-1].split("_"):
            obj.traditional_ChangeClfModelPath(clfModel_filename)
            name_cell = obj.pathToClfModel.split("/")[-1].split("_")[5:]
            if name_cell[-3] == "WRul":
                obj.include_rules = True
            else:
                obj.include_rules = False
            # print(obj.pathToTokenizer, obj.pathToClfModel)
            # print(name_cell)
            # print(name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1])
            X, _ = obj.traditional_getFeatures(Vect)
            clf_model = obj.traditional_loadClf()
            y_pred = obj.get_predictions(clf_model, X)
            name_cell = name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1]
            print(name_cell)
            NotProcessedDataWithPredictions[name_cell] = y_pred.tolist()
            ProcessedDataWithPredictions[name_cell] = y_pred.tolist()
     
                 

notProcessedDatasetName = generateNameFromDataSetName(obj.path_to_nonPreprocessedDataset.split("."))
NotProcessedDataWithPredictions.to_csv(notProcessedDatasetName)
# processedDatasetName = generateNameFromDataSetName(obj.path_to_PreprocessedDataset.split("."))
# ProcessedDataWithPredictions.to_csv(processedDatasetName)
print("Finished")

++++++++++++++++++++++++++++++++++++++++
Unigrams_WRul_CountVect_Freq-3803_DT
Unigrams_WRul_CountVect_Freq-3803_GB
Unigrams_WRul_CountVect_Freq-3803_LR
Unigrams_WRul_CountVect_Freq-3803_MLP
Unigrams_WRul_CountVect_Freq-3803_NB
Unigrams_WRul_CountVect_Freq-3803_RF
Unigrams_WRul_CountVect_Freq-3803_SVM
++++++++++++++++++++++++++++++++++++++++
UniAndBigrams_WRul_CountVect_Freq-16176_DT
UniAndBigrams_WRul_CountVect_Freq-16176_GB
UniAndBigrams_WRul_CountVect_Freq-16176_LR
UniAndBigrams_WRul_CountVect_Freq-16176_MLP
UniAndBigrams_WRul_CountVect_Freq-16176_NB
UniAndBigrams_WRul_CountVect_Freq-16176_RF
UniAndBigrams_WRul_CountVect_Freq-16176_SVM
++++++++++++++++++++++++++++++++++++++++
UniBiAndTrigrams_WRul_CountVect_Freq-32026_DT
UniBiAndTrigrams_WRul_CountVect_Freq-32026_GB
UniBiAndTrigrams_WRul_CountVect_Freq-32026_LR
UniBiAndTrigrams_WRul_CountVect_Freq-32026_MLP
UniBiAndTrigrams_WRul_CountVect_Freq-32026_NB
UniBiAndTrigrams_WRul_CountVect_Freq-32026_RF
UniBiAndTrigrams_WRul_CountVect_Freq

In [83]:
pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample_withPredictions.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed_withPredictions.csv"
pathToVectFolder = "../tfidf_vect/"
pathToClfFolder = "../tfidf_clfModels/"


obj = UsePretrainedVectClf(pathToVectFolder, pathToClfModel, path_to_PreprocessedDataset, path_to_nonPreprocessedDataset, include_rules = False)
NotProcessedDataWithPredictions = pd.read_csv(obj.path_to_nonPreprocessedDataset, encoding = "ISO-8859-1")
ProcessedDataWithPredictions = pd.read_csv(obj.path_to_PreprocessedDataset, encoding = "ISO-8859-1")
# print(NotProcessedDataWithPredictions.shape, ProcessedDataWithPredictions.shape)
for vect_filename in glob.glob(pathToVectFolder + "/*"):
    print(40*"+")
    obj.traditional_ChangeVectPath(vect_filename)
    Vect = obj.traditional_loadVect()
    for clfModel_filename in glob.glob(pathToClfFolder + "*"):
        if clfModel_filename.split("/")[-1].split("_")[5:-1] == vect_filename.split("/")[-1].split("_"):
            obj.traditional_ChangeClfModelPath(clfModel_filename)
            name_cell = obj.pathToClfModel.split("/")[-1].split("_")[5:]
            if name_cell[-3] == "WRul":
                obj.include_rules = True
            else:
                obj.include_rules = False
            # print(obj.pathToTokenizer, obj.pathToClfModel)
            # print(name_cell)
            # print(name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1])
            X, _ = obj.traditional_getFeatures(Vect)
            clf_model = obj.traditional_loadClf()
            y_pred = obj.get_predictions(clf_model, X)
            name_cell = name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1]
            print(name_cell)
            NotProcessedDataWithPredictions[name_cell] = y_pred.tolist()
            ProcessedDataWithPredictions[name_cell] = y_pred.tolist()
     
                 

notProcessedDatasetName = generateNameFromDataSetName(obj.path_to_nonPreprocessedDataset.split("."))
NotProcessedDataWithPredictions.to_csv(notProcessedDatasetName)
# processedDatasetName = generateNameFromDataSetName(obj.path_to_PreprocessedDataset.split("."))
# ProcessedDataWithPredictions.to_csv(processedDatasetName)
print("Finished")

++++++++++++++++++++++++++++++++++++++++
Unigrams_WoRul_TfIdfVect_Freq-3785_DT
Unigrams_WoRul_TfIdfVect_Freq-3785_GB
Unigrams_WoRul_TfIdfVect_Freq-3785_LR
Unigrams_WoRul_TfIdfVect_Freq-3785_MLP
Unigrams_WoRul_TfIdfVect_Freq-3785_NB
Unigrams_WoRul_TfIdfVect_Freq-3785_RF
Unigrams_WoRul_TfIdfVect_Freq-3785_SVM
++++++++++++++++++++++++++++++++++++++++
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_DT
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_GB
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_LR
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_MLP
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_NB
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_RF
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_SVM
++++++++++++++++++++++++++++++++++++++++
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_DT
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_GB
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_LR
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_MLP
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_NB
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_RF
UniBiAndTrigrams

In [88]:
path = "../datasets/500_random_sample_withPredictions_withPredictions.csv"
df = pd.read_csv(path , encoding = "ISO-8859-1")
df.iloc[:, 6:].shape

(500, 168)

In [78]:
# import glob



# path_to_nonPreprocessedDataset = "../datasets/500_random_sample (copy).csv"
# print(glob.glob(pathToVectFolder + "/*"))

# df['DT'] = y_bin_pred.tolist()
# df['GB'] = np.random.randint(2, size=500).tolist()
# df['MLP'] = np.random.randint(2, size=500).tolist()
# df['RF'] = np.random.randint(2, size=500).tolist() 

++++++++++++++++++++++++++++++++++++++++
Unigrams_WoRul_TfIdfVect_Freq-3785_DT
Unigrams_WoRul_TfIdfVect_Freq-3785_GB
Unigrams_WoRul_TfIdfVect_Freq-3785_LR
Unigrams_WoRul_TfIdfVect_Freq-3785_MLP
Unigrams_WoRul_TfIdfVect_Freq-3785_NB
Unigrams_WoRul_TfIdfVect_Freq-3785_RF
Unigrams_WoRul_TfIdfVect_Freq-3785_SVM
++++++++++++++++++++++++++++++++++++++++
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_DT
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_GB
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_LR
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_MLP
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_NB
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_RF
UniAndBigrams_WoRul_TfIdfVect_Freq-16158_SVM
++++++++++++++++++++++++++++++++++++++++
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_DT
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_GB
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_LR
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_MLP
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_NB
UniBiAndTrigrams_WoRul_TfIdfVect_Freq-32008_RF
UniBiAndTrigrams

In [98]:
path = "../datasets/disaster data.csv"
df = pd.read_csv(path , encoding = "ISO-8859-1")
df.shape
# df.iloc[:, 6:]

/home/irfan/virEnv375/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1287972, 3)

In [97]:
pwd

'/media/irfan/새 볼륨/development/request_identification_ML/PretrainedTokenizerClf'

In [117]:
def get_max_from_rows_into_column(df, indexes):
    """ The function takes dataframe and index of columns, and 
    then it returns the array containg the most frequent value against each row.
    Example: 
    dataframe:
    'c1' | 'c2'| 'c3' | 'c4'
    -------------------------
      1  |  1  |   1  |  0  
      2  |  1  |   2  |  2
      2  |  3  |   3  |  0 
      2  |  0  |   0  |  0 
      returned array:
      (1, 2, 3, 0)
      df: dataframe
      indexes: indexes of columns
      example: get_max_from_rows_into_column(dataframe, -3:)
      """
    def get_array(df):
        max_unique_label_column = np.empty([0])
        for v in df.values:
            unique_label, freq_label = np.unique(v, return_counts=True)
            # print(unique_label[np.argmax(freq_label)])
            max_unique_label_column = np.append(max_unique_label_column, unique_label[np.argmax(freq_label)])
            
        return max_unique_label_column
    if ":" in indexes:
        if indexes[0] == ":":
            
            df = df.iloc[:, :int(indexes[1:])]
            max_unique_label_column = get_array(df)
        if indexes[-1] == ":":
            df = df.iloc[:, int(indexes[:-1]):]
            max_unique_label_column = get_array(df)
           
        else:
            indexes = [int(x) for x in indexes.split(":")]
            df = df.iloc[:, indexes[0]: indexes[1]]
            max_unique_label_column = get_array(df)
    else:
            indexes = [int(x) for x in indexes.split(",")]
            df = df.iloc[:, indexes]
            max_unique_label_column = get_array(df)
    return max_unique_label_column.astype(int)
#     for v in df.iloc[:, indexes].values:
#         unique_label, freq_label = np.unique(v, return_counts=True)
#         print(unique_label)
#         print(freq_label)
#         print(unique_label[np.argmax(freq_label)])
#     #     print(counts.indexOf(max(counts)))
    #     print(unique)

#         print(40*"*")

d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 3, 0], 'c4': [0, 2, 0, 0]}
# path = "../datasets/500_random_sample_withPredictions_withPredictions.csv"
# df = pd.read_csv(path , encoding = "ISO-8859-1")
df = pd.DataFrame(data=d)

indexes = "2:"
df["tweet_class"] = get_max_from_rows_into_column(df, indexes)
df

,c1,c2,c3,c4,tweet_class
0,1,1,1,0,0
1,2,1,2,2,2
2,2,3,3,0,0
3,2,0,0,0,0


In [96]:
def get_max_from_rows_into_column(df, indexes):
    """ The function takes dataframe and index of columns, and 
    then it returns the array containg the most frequent value against each row.
    Example: 
    dataframe:
    'c1' | 'c2'| 'c3' | 'c4'
    -------------------------
      1  |  1  |   1  |  0  
      2  |  1  |   2  |  2
      2  |  3  |   3  |  0 
      2  |  0  |   0  |  0 
      returned array:
      (1, 2, 3, 0)
      df: dataframe
      indexes: indexes of columns
      example: get_max_from_rows_into_column(dataframe, -3:)
      """
    def get_array(df):
        max_unique_label_column = np.empty([0])
        for v in df.values:
            unique_label, freq_label = np.unique(v, return_counts=True)
            # print(unique_label[np.argmax(freq_label)])
            max_unique_label_column = np.append(max_unique_label_column, unique_label[np.argmax(freq_label)])
            
        return max_unique_label_column
    if ":" in indexes:
        if indexes[0] == ":":
            
            df = df.iloc[:, :int(indexes[1:])]
            max_unique_label_column = get_array(df)
        if indexes[-1] == ":":
            df = df.iloc[:, int(indexes[:-1]):]
            max_unique_label_column = get_array(df)
           
        else:
            indexes = [int(x) for x in indexes.split(":")]
            df = df.iloc[:, indexes[0]: indexes[1]]
            max_unique_label_column = get_array(df)
    else:
            indexes = [int(x) for x in indexes.split(",")]
            df = df.iloc[:, indexes]
            max_unique_label_column = get_array(df)
    return max_unique_label_column.astype(int)


path = "500_random_sample_withPredictions_withPredictions_withPredictions.csv"
df = pd.read_csv(path,  encoding = "ISO-8859-1")

indexes = "4:"
df["tweet_class"] = get_max_from_rows_into_column(df, indexes)
df = df[["tweetID", "tweet_text", "tweet_class"]]
df.to_csv("disaster data.csv", index=)

0.0    434
1.0     66
Name: new, dtype: int64

In [31]:
import numpy as np

df['DT'] = y_bin_pred.tolist()
df['GB'] = np.random.randint(2, size=500).tolist()
df['MLP'] = np.random.randint(2, size=500).tolist()
df['RF'] = np.random.randint(2, size=500).tolist()

In [134]:
df["request"] = df.iloc[:, -4:].mean(axis=1) > 0.6

In [176]:
df.iloc[:, -4:]

,DT,GB,MLP,RF
0,0,0,1,0
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,0,0
...,...,...,...,...
495,1,0,0,1
496,0,0,1,1
497,0,0,1,1
498,0,1,0,1


In [358]:
e = np.empty([0])
e = np.append(e, 1)
e = np.append(e, 2)
e = np.append(e, 3)
print(list(e))

[1.0, 2.0, 3.0]


In [345]:
df.iloc[:, -2:].values

array([[1, 0],
       [2, 2],
       [3, 0],
       [0, 0]], dtype=int64)

In [171]:
df.iloc[:, -:]

,GB,MLP,RF,num_uniq
0,0,1,1,4
1,0,0,0,4
2,0,0,0,4
3,1,1,0,4
4,1,1,1,4
...,...,...,...,...
495,0,0,0,4
496,1,0,1,4
497,0,0,0,4
498,0,0,0,4


1
2
3
0
[1. 2. 3. 0.]


,c1,c2,c3,c4,new
0,1,1,1,0,1.0
1,2,1,2,2,2.0
2,2,3,3,0,3.0
3,2,0,0,0,0.0


In [300]:
indexes = "-4:-3"
print(indexes)
if ":" in indexes:
    if indexes[0] == ":" or indexes[-1] == ":":
        indexes = indexes
    else:
        indexes = [int(x) for x in indexes.split(":")]
        return
else:
    indexes = [int(x) for x in indexes.split(",")]
print(indexes)
df.iloc[:, indexes]


-4:-3
[-4, -3]


,DT,GB
0,0,0
1,1,0
2,0,1
3,0,1
4,0,1
...,...,...
495,1,0
496,0,0
497,0,0
498,0,1


In [335]:
d = {'c1': [1, 2, 2, 2], 'c2': [1, 1, 3, 0], 'c3': [1, 2, 2, 0], 'c4': [0, 2, 0, 0]}
df = pd.DataFrame(data=d)
df

,c1,c2,c3,c4
0,1,1,1,0
1,2,1,2,0
2,2,3,2,0
3,2,0,0,0


In [299]:
[int(x) for x in indexes.split(":")]

ValueError: invalid literal for int() with base 10: ''

In [323]:
indexes = ":-433"
indexes[1:]

'-433'

In [307]:
df.iloc[:, -4]

0      0
1      1
2      0
3      0
4      0
      ..
495    1
496    0
497    0
498    0
499    0
Name: DT, Length: 500, dtype: int64

In [225]:
df.iloc[:, -4:]

,DT,GB,MLP,RF
0,0,0,1,0
1,1,0,1,1
2,0,1,1,1
3,0,1,1,1
4,0,1,0,0
...,...,...,...,...
495,1,0,0,1
496,0,0,1,1
497,0,0,1,1
498,0,1,0,1


In [ ]:
df['num_uniq'] = [len(set(v[pd.notna(v)].tolist())) for v in df.iloc[:, -4:].values]

In [166]:
df['num_uniq'] = df.iloc[:, -4:].stack().groupby(level=0).count()

In [167]:
unique, counts = np.unique(array, return_counts=True)

0      4
1      4
2      4
3      4
4      4
      ..
495    4
496    4
497    4
498    4
499    4
Name: num_uniq, Length: 500, dtype: int64

In [193]:
arr = np.array([11, 12, 13, 14, 15, 16, 17, 15, 11, 12, 14, 15, 16, 17])

In [194]:
arr

array([11, 12, 13, 14, 15, 16, 17, 15, 11, 12, 14, 15, 16, 17])

In [16]:
pwd

'/media/irfan/새 볼륨/development/request_identification_ML/PretrainedTokenizerClf'